In [45]:
import requests
from bs4 import BeautifulSoup

In [42]:
# post with payload
def postToCointelegraph(pageLength, offest):
    payload = {
        "operationName": "TagPageQuery",
        "query": "query TagPageQuery($short: String, $slug: String!, $order: String, $offset: Int!, $length: Int!) {\n  locale(short: $short) {\n    tag(slug: $slug) {\n      id\n      slug\n      avatar\n      createdAt\n      updatedAt\n      redirectRelativeUrl\n      alternates {\n        short\n        domain\n        id\n        code\n        __typename\n      }\n      tagTranslates {\n        id\n        title\n        metaTitle\n        pageTitle\n        description\n        metaDescription\n        keywords\n        __typename\n      }\n      posts(order: $order, offset: $offset, length: $length) {\n        data {\n          id\n          slug\n          views\n          postTranslate {\n            id\n            title\n            avatar\n            published\n            publishedHumanFormat\n            leadText\n            author {\n              id\n              slug\n              authorTranslates {\n                id\n                name\n                __typename\n              }\n              __typename\n            }\n            __typename\n          }\n          category {\n            id\n            slug\n            __typename\n          }\n          author {\n            id\n            slug\n            authorTranslates {\n              id\n              name\n              __typename\n            }\n            __typename\n          }\n          postBadge {\n            id\n            label\n            postBadgeTranslates {\n              id\n              title\n              __typename\n            }\n            __typename\n          }\n          showShares\n          showStats\n          __typename\n        }\n        postsCount\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}",
        "variables": {
            "cacheTimeInMS": 300000,
            "length": pageLength,
            "offset": offest,
            "order": "postPublishedTime",
            "short": "en",
            "slug": "markets"
        }
    }
    url = "https://conpletus.cointelegraph.com/v1/"
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

In [47]:
pageLength = 15
limit = 100

newsBasePath = "https://cointelegraph.com/news"

newsList = []
for i in range(0, limit, pageLength):
    data = postToCointelegraph(pageLength, i)
    posts = data["data"]["locale"]["tag"]["posts"]["data"]
    if len(posts) == 0:
        break

    for post in posts:
        postSlug = post["slug"]
        newsList.append(f"{newsBasePath}/{postSlug}")


In [59]:
articles = []

for news in newsList:
    # crawl news with tag article
    response = requests.get(news, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.content, "html.parser")
    article = soup.find("article")

    # find h1 tag class post__title
    title = article.find("h1", class_="post__title").text

    # find all p tags
    paragraphs = article.find_all("p")
    # get p tag text
    content = "\n".join([p.text for p in paragraphs])

    # find li tag class tags-list__item
    tags = article.find_all("li", class_="tags-list__item")
    # get a tag text
    hashtags = [tag.find("a").text for tag in tags]

    articles.append({
        "title": title,
        "content": content,
        "hashtags": hashtags
    })

In [63]:
# save articles to json file
import json
with open("articles.json", "w") as file:
    json.dump(articles, file, indent=4)